# Multi Environment Deployment (CN)

## What is Multi Environment

在经典生产级项目中, 通常会使用一个中心化的 DevOps 环境, 以及多个 Workload 环境来部署 App. DevOps 环境一般不部署真正的 App, 而是专注管理运维所用的资源, 例如 CI Job runtime, 以及部署所用到的 Artifacts. 而 Workload 环境则是真正用来部署 App 的. 而将这些环境放在不同的 AWS Account 上的好处是便于给团队中的不同人员不同的权限, 便于管理. 例如管理员可以管理 DevOps 上的全部资源. 每个 App 的负责人可以编辑 Production, 而普通开发者只能编辑 Sandbox, 而不能编辑 Production.

在后续的文档中, 我们会用 **1 + N** 这个术语来指代一个 DevOps 环境以及多个 Workload 环境的组合.

## What is DevOps Environment

如前面所说, DevOps 环境是用来跑 CI/CD 的自动化代码的, 以及上传部署所用到的 Artifacts. 而部署的任务则是把 App 部署到 Workload 环境中. 这就设计到跨环境权限管理问题:

1. DevOps 环境要有对 Workload 环境操作的能力. 这一点通常用 assume role 技术解决 (让 DevOps 上的 Role assume Workload 上的 Role).
2. Workload 环境要有能从 DevOps 环境中 pull artifacts 的能力. 这一点通常用 Resource Policy 技术解决. 例如 AWS S3 bucket policy.

本项目要求必须有 **devops** 环境. **devops** 环境和其他 workload 环境可以用不同的 AWS Account 隔离, 也可以在同一个 AWS Account 内用命名空间进行软隔离.

## How many Workload Environments do I need

对于不同的项目, 一共需要多少个环境是要视具体情况而定的. 但是通常情况下, **一般都至少要三个 Workload 环境**, **sbx** 是完全隔离的, 用于开发. **tst** 是用于 integration test 和 end to end test 的. 而 **prd** 则是真正的生产环境. 对于有些项目还可能会有 stg 用于克隆生产环境中的数据, 用真实数据进行测试, 也可能有 qa 用户质量检查.

本项目要求至少有 **sbx**, **prd** 两个环境 (这个项目就是为 multi-environments 设计的, 如果只有一个 workload 环境就用不到这个项目了). 用户按照需要可以增加更多的 Workload 环境. 无论用户最终使用了多少个 Workload 环境, 本项目提供了的工具都能使得这些 Workload 环境中的特殊规则可以轻易的和 App 代码集成.

## How many AWS Accounts do I need

在 AWS 官方文档中, 推荐给每一个环境一个单独的 AWS Account, 这样有助于权限隔离. 但是对于有的团队和项目, 是有将这些环境全部都放在一个 Account, 或是将几个环境 (例如把 non-prod 的几个环境放在一个 Account, 而 production 单独一个) 放在一个 Account 中的需求的. 这种情况下的隔离通常是通过 resource naming convention 以及 IAM role 权限实现的.

本项目不对这一点做任何限制, 用户可以使用任何策略. 本项目提供了一个抽象层, 用户只需要跟本项目的框架集成即可.

## How to Define My Project Multi-Environment Setup

如果使用本框架, 我推荐在你的项目代码中创建一个 ``multi_env.py`` 模块, 用于定义你项目所使用的环境的枚举. 当然, 你需要遵循限制, 至少定义 ``devops``, ``sbx``, ``prd`` 三个环境. 在这个例子中, 我们除了必须的三个环境, 还定义了 ``tst``, ``stg`` 两个环境. 示例代码如下.

In [3]:
# content of multi_env.py

import aws_ops_alpha.api as aws_ops_alpha

class EnvNameEnum(aws_ops_alpha.BaseEnvNameEnum):
    """
    Environment enumeration for your project.
    """

    devops = aws_ops_alpha.CommonEnvNameEnum.devops.value
    sbx = aws_ops_alpha.CommonEnvNameEnum.sbx.value
    tst = aws_ops_alpha.CommonEnvNameEnum.tst.value
    stg = aws_ops_alpha.CommonEnvNameEnum.stg.value
    prd = aws_ops_alpha.CommonEnvNameEnum.prd.value

## What is the Current Environment

你的 DevOps 业务逻辑中肯定会有许多需要根据 environment name 来对某种业务流程进行稍许修改. 例如在一个 devops 环境中你可能会从本地配置文件中读取 config, 而在 production 环境中你会从 environment variable 中读取 config. 在你调用这些业务逻辑的时候, 只需要像这样 ``EnvNameEnum.devops`` 引用不同的 ``env_name`` 即可. 但是这种 DevOps 的业务逻辑一般是非常多且复杂的, 在代码中反复的像这样 ``EnvNameEnum.devops`` 对某个特定值进行引用是很容易出错的, 且后期不易修改, 不好维护.

为了解决这一问题, 我们引入了 current environment 的概念. 我们可以创建一个函数 ``detect_current_env``, 这个函数能根据当前所在的 runtime, environment variables 中的变量值, 以及一些 context data 来进行判断, 用程序来检测我们在执行 DevOps 业务逻辑时应该切换到哪个环境. 举例来说, 在 local 本地开发 runtime 下, 如果没有特殊指定, 我们默认使用 sandbox 环境进行开发. 而在 CI runtime 下, 我们可以用 ``USER_ENV_NAME`` 这个环境变量来指定该用哪个环境 (这只是个例子, 你可以用不同的方法), 而在 App 的 runtime 下, 例如 EC2 可以用本地的配置文件来读取环境, Lambda Function 可以用 environment variables 来读. 总之在特定项目中是必然有一套约定的, 所以我们只要将这个约定用 ``detect_current_env`` 函数实现了即可. 这样避免了重复的条件判断逻辑在代码库中到处都是, 大大提高了代码的可读性和可维护性.

最终你的 ``multi_env.py`` 代码看起来应该像这样.

In [ ]:
# content of multi_env.py

import aws_ops_alpha.api as aws_ops_alpha
from .your_project_runtime_module import runtime


class EnvNameEnum(aws_ops_alpha.BaseEnvNameEnum):
    """
    Environment enumeration for your project.
    """

    devops = aws_ops_alpha.CommonEnvNameEnum.devops.value
    sbx = aws_ops_alpha.CommonEnvNameEnum.sbx.value
    tst = aws_ops_alpha.CommonEnvNameEnum.tst.value
    stg = aws_ops_alpha.CommonEnvNameEnum.stg.value
    prd = aws_ops_alpha.CommonEnvNameEnum.prd.value


def detect_current_env() -> str:
    # ----------------------------------------------------------------------
    # you can uncomment this line to force to use certain env
    # from your local laptop to run application code, tests, ...
    # ----------------------------------------------------------------------
    # return EnvNameEnum.prd.value

    # use the aws_ops_alpha recommended setup
    return aws_ops_alpha.detect_current_env(runtime, EnvNameEnum)

而你的其他代码如果要使用 ``multi_env.py`` 模块, 看起来就像是这样. 下面我们给出了一个简单示例, 用于读取 config 数据.

In [4]:
import typing as T
from path.to.multi_env import EnvNameEnum, detect_current_env

def load_config(
    env_name: T.Optional[str] = None,
):
    if env_name is None:
        env_name = detect_current_env()
    # then load config data of the given environment name